In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
proj_path = '/content/drive/MyDrive/Noise_Dataset'
os.chdir(proj_path)
os.getcwd()
print(os.listdir(proj_path))

['train', 'test']


In [ ]:
import os
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1TTPWq-skmu9iLiFvrLU-8qUL4vrvacC0/Noise_Dataset'

In [ ]:
from sklearn.datasets import make_blobs
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
#from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.optimizers import SGD
#from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model

In [ ]:
#image sizing
IMAGE_SIZE=[224,224]
train_path='/content/drive/MyDrive/Noise_Dataset/train'
valid_path='/content/drive/MyDrive/Noise_Dataset/test'

# Preprocessing the Training set
#train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
train_datagen = ImageDataGenerator(rescale = 1./255, dtype = 'float32', preprocessing_function=tf.keras.applications.resnet.preprocess_input)
test_datagen = ImageDataGenerator(rescale = 1./255, dtype = 'float32', preprocessing_function=tf.keras.applications.resnet.preprocess_input)

# Preprocessing the Train set
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
# Preprocessing the Test set
#test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 628 images belonging to 4 classes.
Found 272 images belonging to 4 classes.


In [ ]:
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.resnet import ResNet101
import tensorflow as tf

IMAGE_SIZE=[224,224,3]

def model_resnet(summary = True, classes = 4):
  resnet=ResNet101(input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
  for layer in resnet.layers:
    layer.trainable=False

  x = Flatten()(resnet.output)
  prediction = Dense(classes, activation='softmax')(x)

  # create a model object
  model = Model(inputs=resnet.input, outputs=prediction)
  if summary == True:
    model.summary()
  else:
    pass
  
  return model

In [ ]:
##for layer in resnet.layers:
  ##layer.trainable=False

In [ ]:
#classes=4
#model= tf.keras.models.Sequential()
#model.add(resnet) 
#model.add(Dense(classes))
#model.summary()

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0005,
    name="Adam"),
  metrics=['accuracy']
)

In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20


ValueError: ignored

In [ ]:
import tensorflow as tf
model.save('model101.h5')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize = (12,8))
plt.subplot(1,2,1)
plt.plot(r.history['loss'])
T_loss = r.history['loss']
plt.plot(r.history['val_loss'])
V_loss = r.history['val_loss']
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='best')
  
plt.subplot(1,2,2)
plt.plot(r.history['accuracy'])
T_acc = r.history['accuracy']
plt.plot(r.history['val_accuracy'])
V_acc = r.history['val_accuracy']
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='best')